In [1]:
import pandas as pd
import os
from ast import literal_eval
from tqdm import tqdm

In [2]:
DATA_PATH = os.path.join(
    '..', '..', '..', "data", "frameworks_data", 'data_v0.7.1'
)

full_data = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset_with_translations.csv'))
test_data = pd.read_csv(os.path.join(DATA_PATH, 'test_v0.7.1.csv'))

/home/selim/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
train_val_full = full_data[~full_data.entry_id.isin(test_data.entry_id)]

In [4]:
fr_df = train_val_full[['entry_id', 'translation_fr']]\
        .rename(columns={'translation_fr':'excerpt'}).dropna()
en_df = train_val_full[['entry_id', 'translation_en']]\
        .rename(columns={'translation_en':'excerpt'}).dropna()
es_df = train_val_full[['entry_id', 'translation_es']]\
        .rename(columns={'translation_es':'excerpt'}).dropna()

augmented_data = pd.concat([en_df, fr_df, es_df])

In [5]:
augmented_data = pd.merge(
    right=train_val_full.drop(columns=[
        'excerpt', 'translation_en', 'translation_fr', 'translation_es']
                           ),
    left=augmented_data[['entry_id', 'excerpt']],
    on='entry_id',
    how='right'
)

In [6]:
train_val_df = pd.concat([train_val_full, augmented_data]).drop(columns=[
        'translation_en', 'translation_fr', 'translation_es', 'lang']
                           )

In [7]:
train_val_df.columns

Index(['entry_id', 'excerpt', 'analysis_framework_id', 'lead_id', 'project_id',
       'verified', 'sectors', 'subpillars_2d', 'subpillars_1d', 'geo_location',
       'specific_needs_groups', 'severity', 'info_date', 'reliability',
       'affected_groups_level_0', 'affected_groups_level_1',
       'affected_groups_level_2', 'affected_groups_level_3', 'age', 'gender',
       'source_type', 'url', 'website'],
      dtype='object')

In [8]:
train_val_df.shape

(422327, 23)

In [9]:
train_val_df['present_prim_tags'] = train_val_df.apply(
    lambda x: [column for column in ['sectors','subpillars_2d', 'subpillars_1d'] if len(x[column])>2], axis=1
               )

In [10]:
train_val_df['pillars_1d'] = train_val_df.subpillars_1d.apply(
    lambda x: [item.split('->')[0] for item in literal_eval(x)]
               )

In [11]:
train_val_df['pillars_2d'] = train_val_df.subpillars_2d.apply(
    lambda x: [item.split('->')[0] for item in literal_eval(x)]
               )

In [12]:
train_val_df['impact_capresp_humcond'] = train_val_df.subpillars_2d.apply(
    lambda x: [
        item for item in literal_eval(x) if item.split('->')[0] in [
            'Capacities & Response', 'Humanitarian Conditions', 'Impact'
        ]
    ]
)

In [13]:
train_val_df['need_intervention_risk'] = train_val_df.subpillars_2d.apply(
    lambda x: [
        item for item in literal_eval(x) if item.split('->')[0] in [
            'At Risk', 'Priority Needs', 'Priority Interventions'
        ]
    ]
)

In [14]:
train_val_df['context_covid'] = train_val_df.subpillars_1d.apply(
    lambda x: [
        item for item in literal_eval(x) if item.split('->')[0] in [
            'Context', 'Covid-19'
        ]
    ]
)

In [15]:
train_val_df['displacement_shockevent'] = train_val_df.subpillars_1d.apply(
    lambda x: [
        item for item in literal_eval(x) if item.split('->')[0] in [
            'Displacement', 'Shock/Event'
        ]
    ]
)

In [16]:
train_val_df['access_infcom_casualities'] = train_val_df.subpillars_1d.apply(
    lambda x: [
        item for item in literal_eval(x) if item.split('->')[0] in [
            'Humanitarian Access', 'Information And Communication', 'Casualties'
        ]
    ]
)

In [15]:
data.to_csv(os.path.join(DATA_PATH, 'prim_tags.csv'))

In [17]:
train_val_df['present_prim_tags'].apply(lambda x: len(x)>0).sum()

401460